In [16]:
import numpy as np
import pandas as pd
import plotly.plotly as py
import plotly.graph_objs as go

In [8]:
%load_ext sql
%config SqlMagic.autopandas=True
%matplotlib inline

# %qtconsole

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [6]:
%sql sqlite:///data/sales.db

u'Connected: None@data/sales.db'

### Some usefule SQL queries

_Note that sqlite3 is quite limited as a sql db. e.g. it doesn't support variables and a lot of functions (mean, std..) offered by other db (MySQL, postgreSQL, etc.). So queries below are simple, or I use python to access more complex information.

#### Get unique entries for all dates (either new, or sold cars)
```SQL
select count(line_id) ss, min(line_id), * 
from salesVR 
group by year, make, model, trim, mileage 
having ss = 1 
order by make, model, trim, year;
```

#### Get unique entries for all dates (either new, or sold cars)
```SQL
select date, count(ss) from
(select count(line_id) ss, min(line_id), * 
from salesVR 
group by year, make, model, trim, mileage 
having ss = 1) A
group by date
;
```
        * 2016-09-12  260 (cars sold in 3 days..?)
        * 2016-09-15  250 (new cars added to catalog in 3 days?)
   

## See what cars have been sold and added each days

I'm assuming, perhaps simplistically, that cars taken of of the catalog have been sold. 

## Plot make average cost 
First get list of individual cars, then average per make. Most cars will show up every day because they haven't been sold. The price for some of these cars are changing so I first take the average over the 4 days for each cars.

To find unique cars I group by year, make, model, trim, mileage. This assumes that mileage doesn't change but that price can. 

```SQL
select make, count(make) num_cars, sum(price)/count(price) ave_price, 
        max(price) max_price, min(price) min_price
from 
(select make, sum(price)/count(price) price from salesVR
group by year, make, model, trim, mileage) DAY
group by make
order by ave_price;
```

In [17]:
make_ave_price = %sql select make, count(make) num_cars, sum(price)/count(price) ave_price from (select make, sum(price)/count(price) price from salesVR group by year, make, model, trim, mileage) DAY group by make order by ave_price;

data = [go.Bar(
            x=make_ave_price["make"],
            y=make_ave_price["ave_price"]
    )]

py.iplot(data, filename='basic-bar')

Done.
